In [2]:
import os
import requests
import openpyxl
import pandas as pd
import numpy as np
import re
import yaml

In [4]:
def fetch_and_filter_data(url): # Function to fetch and filter the JSON file
    response = requests.get(url)
    json_data = response.json()
    for study in json_data['study']:
        for site in study['site']:
            coordinates = site['geo']['geometry']['coordinates'] # Extract latitude and longitude from resepctive json key
            latitude = coordinates[0]
            longitude = coordinates[1]

            file_url_list = []
            file_var = []
            for paleo_data in site['paleoData']:
                for data_file in paleo_data['dataFile']:
                    file_url = data_file['fileUrl'] # Extract file URL
                    file_url_list.append(file_url)
                    
                    var_names = [var['cvShortName'] for var in data_file['variables']] # Extract variable names
                    file_var.append(var_names)

    return [latitude, longitude, file_url_list, file_var]


In [ ]:
responses = []

In [6]:
khider_response = fetch_and_filter_data("https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=16017")
responses.append(khider_response)

Latitude: 1.4033, Longitude: 119.078
File URL: https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2011/khider2011.txt
Variables: [None, None, None, None, None, None, None]
Latitude: 32.28, Longitude: -118.38
File URL: https://www.ncei.noaa.gov/pub/data/paleo/paleocean/pacific/bhattacharya2022/bhattacharya2022-odp1012.txt
Variables: ['dD_precip', 'dD_precip_err', 'age_ma', 'dD_wax', 'dD_wax_err', 'd13C_wax', 'd13C_wax_err']
Latitude: 23.03, Longitude: -109.03
File URL: https://www.ncei.noaa.gov/pub/data/paleo/paleocean/pacific/bhattacharya2022/bhattacharya2022-dsdp475.txt
Variables: ['age_ma', 'dD_wax', 'dD_wax_err', 'd13C_wax', 'd13C_wax_err', 'dD_precip', 'dD_precip_err']
Latitude: -.693833, Longitude: -85.33325
File URL: https://www.ncei.noaa.gov/pub/data/paleo/paleocean/pacific/glaubke2024/glaubke2024-09mc.txt
Variables: ['TimeSlice', 'Age_kyrs', 'SampSize_ug', 'Mg/Ca', 'Mn/Ca', 'Al/Ca', 'Fe/Ca', 'Mg/Ca-SWT']
Latitude: -.180533, Longitude: -85.86673
File URL: htt

In [ ]:
bhattacharya_response = fetch_and_filter_data("https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=78037")
responses.append(bhattacharya_response)

In [ ]:
glaubek_response = fetch_and_filter_data("https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=80598")
responses.append(glaubek_response)

In [ ]:
cody_response = fetch_and_filter_data("https://www.ncei.noaa.gov/access/paleo-search/study/search.json?xmlId=66868")
responses.append(cody_response)

In [ ]:
def filter_rows_cols(block_str):
    # Find occurrences of 'start..end' patterns
    all_ranges = re.findall(r'(\d+)\.\.(\d+)', block_str)
    
    # Separate row and column ranges for every block
    row_ranges = [(int(start), int(end)) for start, end in all_ranges[::2]]
    col_ranges = [(int(start), int(end)) for start, end in all_ranges[1::2]]
    return row_ranges, col_ranges

In [ ]:
# Function to extract and print data for each block
def extract_for_all_blocks(df, rows_ranges, cols_ranges, spreadsheet_df):
    for i, (row_range, col_range) in enumerate(zip(rows_ranges, cols_ranges)):
        start_row, end_row = row_range
        start_col, end_col = col_range

        block_data = spreadsheet_df.iloc[start_row-1:end_row, start_col:end_col+1]
        print(f"Block {i+1} (rows {start_row} to {end_row}, columns {start_col} to {end_col}):\n")
        print(block_data)
        print("\n" + "="*50 + "\n")

In [8]:
yaml_path = './isi-table-understanding/tmp/output/blue2019dust-corea.xlsx_0_Sheet1.yaml'
excel_path = './isi-table-understanding/tmp/output/blue2019dust-corea.xlsx_colorised.xlsx'

# Load YAML file to check structure
with open(yaml_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

spreadsheet_df = pd.read_excel(excel_path, sheet_name=0)

# Extract the 'ObservationData' from the YAML file
observation_data_str = yaml_data['variables']['data0']['ObservationData']

rows_ranges, cols_ranges = filter_rows_cols(observation_data_str)
# Extract and print data for all blocks
extract_for_all_blocks(spreadsheet_df, rows_ranges, cols_ranges)

Block 1 (rows 1 to 1, columns 0 to 4):

   Unnamed: 0  age_calBP  dustfrac   sedrate     dens
0           0    2722.83   0.33702  0.049261  0.24186


